# SKGB - Semantic Knowledge Graph Builder (H100 GPU + Hugging Face Edition)\n\nThis notebook demonstrates the full **DynamicKGConstruction** pipeline using **Hugging Face** models on **H100 GPU**:\n\n**PDF -> Docling Markdown -> Semantic Chunks -> itext2kg Knowledge Graph -> Visualization**\n\n> **Runtime**: Go to *Runtime -> Change runtime type* and select **H100 GPU** for maximum performance.\n> **Hugging Face**: Uses `meta-llama/Llama-3.1-8B-Instruct` for LLM and `sentence-transformers/all-MiniLM-L6-v2` for embeddings.\n> **Note**: You'll need a Hugging Face token with access to Llama models.

## 1. Hugging Face Authentication\n\nYou need a Hugging Face token to access gated models like Llama.\n1. Get your token from https://huggingface.co/settings/tokens\n2. Add it to Colab secrets or enter it below:

In [ ]:
# Hugging Face Authentication\nimport os\nfrom getpass import getpass\n\n# Try to get token from Colab secrets first\ntry:\n    from google.colab import userdata\n    HF_TOKEN = userdata.get('HF_TOKEN')\n    print("✅ Loaded HF_TOKEN from Colab secrets")\nexcept:\n    HF_TOKEN = None\n\n# If not in secrets, prompt user\nif not HF_TOKEN:\n    print("🔑 Please enter your Hugging Face token:")\n    print("   (Get one at https://huggingface.co/settings/tokens)")\n    HF_TOKEN = getpass("HF_TOKEN: ")\n\nos.environ['HF_TOKEN'] = HF_TOKEN\n\n# Login to Hugging Face\n!huggingface-cli login --token $HF_TOKEN\n\nprint("\n✅ Hugging Face authentication complete!")

## 2. Verify H100 GPU\n\nCheck that we have GPU access and it's an H100:

In [ ]:
# Verify GPU\nimport torch\nimport subprocess\n\nprint("GPU Available:", torch.cuda.is_available())\nprint("GPU Count:", torch.cuda.device_count())\n\nif torch.cuda.is_available():\n    for i in range(torch.cuda.device_count()):\n        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")\n        print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")\n\n# Check with nvidia-smi\nprint("\n=== nvidia-smi output ===")\n!nvidia-smi

## 3. Install Dependencies\n\nInstall SKGB with Hugging Face support:

In [ ]:
# Clone the repository\n!git clone https://github.com/edwinidrus/DynamicKGConstruction.git 2>/dev/null || echo "Already cloned"\n%cd DynamicKGConstruction\n\n# Install base requirements\n!pip install -q -r requirements.txt\n\n# Install Hugging Face specific dependencies\n!pip install -q transformers accelerate sentence-transformers huggingface_hub\n\nprint("\n✅ Dependencies installed!")

## 4. Load Hugging Face Models\n\nLoad the LLM and embeddings models on H100 GPU:

In [ ]:
import torch\nfrom transformers import AutoModelForCausalLM, AutoTokenizer, pipeline\nfrom langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings\n\n# Model configuration for H100\nLLM_MODEL = "meta-llama/Llama-3.1-8B-Instruct"  # ~16GB VRAM\nEMBEDDINGS_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # ~400MB\n\nprint(f"Loading LLM: {LLM_MODEL}")\nprint("This may take 2-3 minutes on first run...\n")\n\n# Load LLM\nllm_tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL, token=os.environ['HF_TOKEN'])\nllm_model = AutoModelForCausalLM.from_pretrained(\n    LLM_MODEL,\n    torch_dtype=torch.bfloat16,\n    device_map="auto",\n    token=os.environ['HF_TOKEN']\n)\n\n# Create pipeline\nllm_pipeline = pipeline(\n    "text-generation",\n    model=llm_model,\n    tokenizer=llm_tokenizer,\n    max_new_tokens=512,\n    temperature=0.0,\n    do_sample=False,\n    return_full_text=False,\n)\n\n# Wrap for LangChain\nllm = HuggingFacePipeline(pipeline=llm_pipeline)\n\nprint(f"\n✅ LLM loaded: {LLM_MODEL}")\nprint(f"   Device: {llm_model.device}")\n\n# Load Embeddings\nprint(f"\nLoading Embeddings: {EMBEDDINGS_MODEL}...")\nembeddings = HuggingFaceEmbeddings(\n    model_name=EMBEDDINGS_MODEL,\n    model_kwargs={"device": "cuda"},\n    encode_kwargs={"normalize_embeddings": True}\n)\n\nprint(f"\n✅ Embeddings loaded: {EMBEDDINGS_MODEL}")\nprint("\n🚀 All models ready on H100 GPU!")

## 5. Apply itext2kg Patch\n\nPatch to handle empty atomic KG lists gracefully:

In [ ]:
# Apply itext2kg patch for IndexError\nimport functools\n\nprint("Applying itext2kg patch...")\n\ntry:\n    import itext2kg.atom.atom as atom_module\n    \n    _original_func = atom_module.Atom.parallel_atomic_merge\n    \n    @functools.wraps(_original_func)\n    def _safe_parallel_atomic_merge(self, kgs, existing_kg=None, rel_threshold=0.7, ent_threshold=0.8, max_workers=8):\n        if not kgs:\n            print("⚠️  No atomic KGs to merge. Returning empty KG.")\n            from itext2kg.graphs.knowledge_graph import KnowledgeGraph\n            return KnowledgeGraph()\n        return _original_func(self, kgs, existing_kg, rel_threshold, ent_threshold, max_workers)\n    \n    atom_module.Atom.parallel_atomic_merge = _safe_parallel_atomic_merge\n    print("✅ Patch applied!")\n    \nexcept Exception as e:\n    print(f"⚠️  Patch not applied: {e}")

## 6. Upload a PDF\n\nUpload your own PDF or use the sample:

In [ ]:
import os\nfrom pathlib import Path\n\nINPUT_DIR = Path("input_docs")\nINPUT_DIR.mkdir(exist_ok=True)\n\n# Option A: Upload from your computer\ntry:\n    from google.colab import files\n    print("Click below to upload a PDF:")\n    uploaded = files.upload()\n    for filename, data in uploaded.items():\n        dest = INPUT_DIR / filename\n        dest.write_bytes(data)\n        print(f"Saved: {dest}")\nexcept ImportError:\n    print("Not in Colab - place PDF in input_docs/ manually")

In [ ]:
# Option B: Download sample PDF (Attention Is All You Need)\nSAMPLE_URL = "https://arxiv.org/pdf/1706.03762"\nSAMPLE_PATH = INPUT_DIR / "attention_is_all_you_need.pdf"\n\nif not SAMPLE_PATH.exists():\n    !wget -q -O "{SAMPLE_PATH}" "{SAMPLE_URL}"\n    print(f"Downloaded: {SAMPLE_PATH}")\nelse:\n    print(f"Sample exists: {SAMPLE_PATH}")\n\npdfs = list(INPUT_DIR.glob("*.pdf"))\nprint(f"\nPDFs available: {[p.name for p in pdfs]}")

## 7. Configure and Run Pipeline\n\nRun the SKGB pipeline with Hugging Face models:

In [ ]:
# Custom Hugging Face adapter for SKGB\nimport asyncio\nimport logging\nfrom typing import Dict, List\nfrom pathlib import Path\n\nlogger = logging.getLogger(__name__)\n\ndef _run(coro):\n    try:\n        return asyncio.run(coro)\n    except RuntimeError:\n        import nest_asyncio\n        nest_asyncio.apply()\n        loop = asyncio.get_event_loop()\n        return loop.run_until_complete(coro)\n\nasync def _build_async_hf(\n    atomic_facts_dict: Dict[str, List[str]],\n    llm,\n    embeddings,\n    ent_threshold: float,\n    rel_threshold: float,\n    max_workers: int,\n):\n    from itext2kg.atom import Atom\n    from itext2kg.graphs.knowledge_graph import KnowledgeGraph\n    \n    total_facts = sum(len(facts) for facts in atomic_facts_dict.values())\n    logger.info(f"Building KG from {total_facts} atomic facts")\n    \n    atom = Atom(llm_model=llm, embeddings_model=embeddings)\n    \n    try:\n        kg = await atom.build_graph_from_different_obs_times(\n            atomic_facts_with_obs_timestamps=atomic_facts_dict,\n            ent_threshold=ent_threshold,\n            rel_threshold=rel_threshold,\n            max_workers=max_workers,\n        )\n    except IndexError as e:\n        if "list index out of range" in str(e):\n            logger.warning("Empty quintuples - returning empty KG")\n            kg = KnowledgeGraph()\n        else:\n            raise\n    \n    return kg\n\nprint("✅ Hugging Face adapter ready")

In [ ]:
# Run the pipeline\nfrom DynamicKGConstruction.skgb.adapters.docling_adapter import docling_convert_to_markdown\nfrom DynamicKGConstruction.skgb.adapters.chunking_adapter import chunk_markdown_files\nfrom DynamicKGConstruction.skgb.export.file_export import export_kg_outputs\nfrom DynamicKGConstruction.skgb.export.neo4j_export import write_neo4j_load_cypher\nimport datetime\nimport json\n\n# Configuration\nOUT_DIR = Path("skgb_output")\nBUILD_DOCLING_DIR = OUT_DIR / "build_docling"\nCHUNKS_DIR = OUT_DIR / "chunks_output"\nKG_DIR = OUT_DIR / "kg_output"\n\nfor d in [OUT_DIR, BUILD_DOCLING_DIR, CHUNKS_DIR, KG_DIR]:\n    d.mkdir(parents=True, exist_ok=True)\n\n# Pipeline parameters\nENT_THRESHOLD = 0.8\nREL_THRESHOLD = 0.7\nMAX_WORKERS = 4  # Higher for H100\n\npdf_path = list(INPUT_DIR.glob("*.pdf"))[0]\nprint(f"Processing: {pdf_path}\n")\n\n# Step 1: Docling\nprint("Step 1: Converting PDF to Markdown...")\nmd_paths = docling_convert_to_markdown(pdf_path, BUILD_DOCLING_DIR)\nprint(f"✅ Created {len(md_paths)} markdown files\n")\n\n# Step 2: Chunking\nprint("Step 2: Creating semantic chunks...")\nchunks = chunk_markdown_files(\n    md_paths=md_paths,\n    min_chunk_words=200,\n    max_chunk_words=800,\n    overlap_words=0,\n)\n\nchunks_json = CHUNKS_DIR / "all_chunks.json"\nchunks_json.write_text(json.dumps(chunks, indent=2), encoding="utf-8")\nprint(f"✅ Created {len(chunks)} chunks\n")\n\n# Step 3: Prepare atomic facts\nprint("Step 3: Preparing atomic facts...")\nt_obs = datetime.datetime.now().strftime("%Y-%m-%d")\natomic_facts_dict = {t_obs: []}\nfor ch in chunks:\n    section = ch.get("section_title", "")\n    content = ch.get("content", "")\n    atomic_facts_dict[t_obs].append(f"[{section}] {content}".strip())\n\n# Step 4: Build KG with Hugging Face\nprint(f"Step 4: Building Knowledge Graph with {LLM_MODEL}...")\nprint("   (This may take several minutes...)\n")\n\nkg = _build_async_hf(\n    atomic_facts_dict=atomic_facts_dict,\n    llm=llm,\n    embeddings=embeddings,\n    ent_threshold=ENT_THRESHOLD,\n    rel_threshold=REL_THRESHOLD,\n    max_workers=MAX_WORKERS,\n)\n\nprint(f"\n✅ KG built: {len(kg.entities)} entities, {len(kg.relations)} relations\n")\n\n# Step 5: Export\nprint("Step 5: Exporting results...")\nexport_kg_outputs(\n    kg=kg,\n    kg_output_dir=KG_DIR,\n    total_chunks=len(chunks),\n    ent_threshold=ENT_THRESHOLD,\n    rel_threshold=REL_THRESHOLD,\n    llm_model=LLM_MODEL,\n    embeddings_model=EMBEDDINGS_MODEL,\n)\n\ncypher_path = write_neo4j_load_cypher(KG_DIR)\nprint(f"✅ Exported to {KG_DIR}\n")\n\nprint("=" * 60)\nprint("🎉 PIPELINE COMPLETE!")\nprint(f"   Markdown: {BUILD_DOCLING_DIR}")\nprint(f"   Chunks:   {chunks_json}")\nprint(f"   KG:       {KG_DIR}")\nprint(f"   Cypher:   {cypher_path}")

## 8. Explore Results\n\nView the generated knowledge graph:

In [ ]:
# List output files\nprint("Output files:")\nfor f in sorted(KG_DIR.rglob("*")):\n    if f.is_file():\n        size = f.stat().st_size\n        print(f"  {f.name:40s} {size:>8,} bytes")

In [ ]:
# View KG JSON\nimport json\n\nkg_data = json.loads((KG_DIR / "knowledge_graph.json").read_text())\nnodes = kg_data.get("nodes", [])\nedges = kg_data.get("edges", [])\n\nprint(f"Nodes: {len(nodes)}, Edges: {len(edges)}\n")\n\nprint("=== Sample Nodes ===")\nfor n in nodes[:10]:\n    print(f"  {n['name'][:40]:40s} ({n.get('label', 'N/A')})")\n\nprint("\n=== Sample Edges ===")\nfor e in edges[:10]:\n    print(f"  {e['source'][:25]} --[{e['relation'][:15]}]--> {e['target'][:25]}")

In [ ]:
# Display interactive visualization\nfrom IPython.display import HTML, display\n\nviz_path = KG_DIR / "kg_visualization.html"\nif viz_path.exists():\n    display(HTML(viz_path.read_text()))\nelse:\n    print("Visualization not found")

## 9. Download Results

In [ ]:
import shutil\n\narchive = shutil.make_archive("skgb_hf_results", "zip", ".", "skgb_output")\nprint(f"Created: {archive}")\n\ntry:\n    from google.colab import files\n    files.download(archive)\nexcept:\n    print(f"Download from: {archive}")